In [2]:
import pandas as pd
import firebase_admin
from firebase_admin import credentials, db
from sklearn.metrics.pairwise import cosine_similarity

if not firebase_admin._apps:
    
    cred = credentials.Certificate('nope.json')
    firebase_admin.initialize_app(cred, {
        'databaseURL': 'https://sistema-rec-dmi-default-rtdb.firebaseio.com/'
    })
ref = db.reference('/base_usuarios') 

datos_firebase = ref.get()

df_firebase = pd.DataFrame(datos_firebase)

rating_columns = [
    'electiva',
    'general_rating',
    'method_rating',
    'personalGrowth_rating',
    'rec_rating',
    'teacher_rating',
    'time_rating'
]

prefixes = ['first', 'second', 'third']

df_ratings_list = []

for indice, fila in df_firebase.iterrows():
    determinante = fila['determinante_electivas']
    
    for i, prefix in enumerate(prefixes, start=1):
        electiva_col = f'{prefix}_electiva'
        general_col = f'{prefix}_general_rating'
        method_col = f'{prefix}_method_rating'
        personalGrowth_col = f'{prefix}_personalGrowth_rating'
        rec_col = f'{prefix}_rec_rating'
        teacher_col = f'{prefix}_teacher_rating'
        time_col = f'{prefix}_time_rating'
        
        electiva = fila[electiva_col]
        
        if electiva.lower() != 'no':
            nueva_fila = {
                'electiva': electiva,
                'general_rating': fila[general_col] if fila[general_col] != 'none' else None,
                'method_rating': fila[method_col] if fila[method_col] != 'none' else None,
                'personalGrowth_rating': fila[personalGrowth_col] if fila[personalGrowth_col] != 'none' else None,
                'rec_rating': fila[rec_col] if fila[rec_col] != 'none' else None,
                'teacher_rating': fila[teacher_col] if fila[teacher_col] != 'none' else None,
                'time_rating': fila[time_col] if fila[time_col] != 'none' else None,
            }
            
            df_temporal = pd.DataFrame([nueva_fila], columns=rating_columns)
            df_ratings_list.append(df_temporal)

df_ratings = pd.concat(df_ratings_list, ignore_index=True)
df_ratings = df_ratings.dropna()

def find_similar_electives(electiva_seleccionada, top_n=3):
    selected_elective_data = df_ratings[df_ratings['electiva'] == electiva_seleccionada]
    selected_elective_data = selected_elective_data.drop(columns=['electiva'])
    similarity_scores = cosine_similarity(selected_elective_data, df_ratings.drop(columns=['electiva']))
    similar_indices = similarity_scores.argsort()[0][-top_n - 1:-1][::-1]
    similar_electives = df_ratings.iloc[similar_indices]['electiva'].tolist()

    return similar_electives


In [ ]:
from flask import Flask, request, jsonify
from flask_cors import CORS

app = Flask(__name__)
CORS(app)

@app.route('/rec-dmi', methods=['POST'])
def recibir_datos():
    datos_recibidos = request.json  
    print(datos_recibidos)
    
    electiva_seleccionada = 'Innovación en servicios'
    similares = find_similar_electives(electiva_seleccionada)
    
    respuesta = {"similares": similares,
                "hello":"hi"}
    
    return jsonify(respuesta)

if __name__ == '__main__':
    app.run(port=5000) 

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [23/Nov/2023 17:53:29] "OPTIONS /rec-dmi HTTP/1.1" 200 -
127.0.0.1 - - [23/Nov/2023 17:53:29] "POST /rec-dmi HTTP/1.1" 200 -


{'prefer': 'dev', 'day': 'martes', 'modal': 'presencial'}


127.0.0.1 - - [23/Nov/2023 19:06:12] "OPTIONS /rec-dmi HTTP/1.1" 200 -
127.0.0.1 - - [23/Nov/2023 19:06:13] "POST /rec-dmi HTTP/1.1" 200 -


{'prefer': 'dev', 'day': 'martes', 'modal': 'presencial', 'importance': 'personal-growth'}


127.0.0.1 - - [23/Nov/2023 19:11:20] "OPTIONS /rec-dmi HTTP/1.1" 200 -
127.0.0.1 - - [23/Nov/2023 19:11:20] "POST /rec-dmi HTTP/1.1" 200 -


{'prefer': 'ui', 'day': 'martes', 'modal': 'virtual', 'importance': 'rec'}
